In [ ]:
# default_exp g_models.vae

# VAE Model

> VAE model

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *

In [ ]:
# export 

class VAE(Encoder_Decoder):
    def __init__(self, encoder, decoder, prior=None, bos_idx=0):
        transition = VAE_Transition(encoder.d_latent)
        super().__init__(encoder, decoder, transition, prior)
            
        self.bos_idx = bos_idx
        
    def forward(self, x, decoder_input=None):
        
        z = self.encoder(x)
        z, kl_loss = self.transition(z)
            
        if decoder_input is None:
            decoder_input = x
            
        output, hiddens, encoded = self.decoder(decoder_input, z)
        return output, kl_loss
    
    def encode(self, x, decoder_input=None):
        z = self.encoder(x)
        z, kl_loss = self.transition(z)
            
        if decoder_input is None:
            decoder_input = x
            
        output, hiddens, encoded = self.decoder(decoder_input, z)
        return encoded
    
    def to_latent(self, x):
        z = self.encoder(x)
        z, kl_loss = self.transition(z)
        return z

    def sample(self, bs, sl, z=None, temperature=1., multinomial=True, z_scale=1.):
        
        preds = idxs = to_device(torch.tensor([self.bos_idx]*bs).long().unsqueeze(-1))
        lps = []
        
        if z is None:
            if self.prior is not None:
                z = to_device(self.prior.rsample([bs]))
            else:
                prior = Normal(torch.zeros((self.encoder.d_latent)),
                               torch.ones((self.encoder.d_latent)))
                z = to_device(self.prior.rsample([bs]))
        
        hiddens = None
        
        for i in range(sl):
            x, hiddens, encoded = self.decoder(idxs, z, hiddens)
            x.div_(temperature)
            
            idxs, lp = x_to_preds(x, multinomial=multinomial)
            
            lps.append(lp)            
            preds = torch.cat([preds, idxs], -1)
            
        return preds[:, 1:], torch.cat(lps,-1)
    
    def sample_no_grad(self, bs, sl, z=None, temperature=1., multinomial=True):
        with torch.no_grad():
            return self.sample(bs, sl, z=z, temperature=temperature, multinomial=multinomial)
        
        
    def get_rl_tensors(self, x, y, temperature=1., latent=None):
        
        if type(x) == list:
            if latent is None:
                latent = self.encoder(x[0])
                z,_ = self.transition(latent)
            else:
                z = latent
            output, hiddens, encoded = self.decoder(x[1], z)
        else:
            if latent is None:
                latent = self.encoder(x)
                z,_ = self.transition(latent)
            else:
                z = latent
            output, hiddens, encoded = self.decoder(x, z)
        
        output.div_(temperature)
        lps = F.log_softmax(output, -1)
        
        if self.prior.trainable:
            prior_lps = self.prior.log_prob(z)
            prior_lps = prior_lps.mean(-1).unsqueeze(-1).unsqueeze(-1)
            pass_through = torch.zeros(prior_lps.shape).float().to(prior_lps.device)
            pass_through = pass_through + prior_lps - prior_lps.detach() # add to gradient chain
            lps = lps + pass_through
            
        lps_gathered = gather_lps(lps, y)
        return output, lps, lps_gathered, encoded
            
    def set_prior_from_stats(self, mu, logvar, trainable=False):
        mu = mu.detach()
        logvar = logvar.detach()
        self.prior = NormalPrior(mu, logvar, trainable)
        
    def set_prior_from_latent(self, z, trainable=False):
        mu, logvar = self.transition.get_stats(z)
        self.set_prior_from_stats(mu, logvar, trainable)
        
    def set_prior_from_encoder(self, x, trainable=False):
        assert x.shape[0]==1, "Must set prior from a single input"
        z = self.encoder(x)
        z = z.squeeze(0)
        self.set_prior_from_latent(z, trainable)

In [ ]:
encoder = LSTM_Encoder(32, 64, 128, 2, 128)
decoder = Conditional_LSTM_Block(32, 64, 128, 64, 128, 2,
                                condition_hidden=True, condition_output=True)
vae = VAE(encoder, decoder)

ints = torch.randint(0, 31, (8, 10))
x = ints[:,:-1]
y = ints[:,1:]

_ = vae(x)

decoder = Conditional_LSTM_Block(32, 64, 128, 64, 128, 2,
                                condition_hidden=False, condition_output=True)
vae = VAE(encoder, decoder)

_ = vae(x)

decoder = Conditional_LSTM_Block(32, 64, 128, 64, 128, 2,
                                condition_hidden=True, condition_output=False)
vae = VAE(encoder, decoder)

_ = vae(x)

_ = vae.sample(8, 16)

z = vae.prior.sample([8])
_ = vae.sample(8, 16, z=z)

o,lp,lpg,e = vae.get_rl_tensors(x,y)

vae.set_prior_from_encoder(x[0].unsqueeze(0), trainable=True);

o,lp,lpg,e = vae.get_rl_tensors(x,y)
loss = lpg.mean()
assert vae.prior.loc.grad is None
loss.backward()
assert vae.prior.loc.grad is not None

latent = torch.randn((x.shape[0], 128))
o,lp,lpg,e = vae.get_rl_tensors(x,y,latent=latent)

In [ ]:
# export

class LSTM_VAE(VAE):
    def __init__(self, d_vocab, d_embedding, d_hidden, n_layers, d_latent,
                enc_drop=0., dec_drop=0., condition_hidden=True, condition_output=True,
                prior=None, bos_idx=0):
        
        encoder = LSTM_Encoder(d_vocab, d_embedding, d_hidden, 
                               n_layers, d_latent, dropout=enc_drop)
        
        decoder = Conditional_LSTM_Block(d_vocab, d_embedding, d_hidden, d_embedding,
                                d_latent, n_layers, lstm_drop=dec_drop, lin_drop=dec_drop, 
                                condition_hidden=condition_hidden, condition_output=condition_output)
        
        super().__init__(encoder, decoder, prior, bos_idx)

In [ ]:
vae = LSTM_VAE(32, 64, 128, 2, 128, condition_hidden=True, condition_output=True)

ints = torch.randint(0, 31, (8, 10))
x = ints[:,:-1]
y = ints[:,1:]

_ = vae(x)

_ = vae.sample(8, 16)

z = vae.prior.sample([8])
_ = vae.sample(8, 16, z=z)

_ = vae.get_rl_tensors(x,y)

In [ ]:
# export

class Conv_VAE(VAE):
    def __init__(self, d_vocab, d_embedding, conv_filters, kernel_sizes, strides, conv_drops,
                 d_hidden, n_layers, d_latent, dec_drop=0., 
                 condition_hidden=True, condition_output=True,
                 prior=None, bos_idx=0):
        
        encoder = Conv_Encoder(d_vocab, d_embedding, d_latent, 
                               conv_filters, kernel_sizes, strides, conv_drops)
        
        decoder = Conditional_LSTM_Block(d_vocab, d_embedding, d_hidden, d_embedding,
                                d_latent, n_layers, lstm_drop=dec_drop, lin_drop=dec_drop, 
                                condition_hidden=condition_hidden, condition_output=condition_output)
        
        super().__init__(encoder, decoder, prior, bos_idx)

In [ ]:
vae = Conv_VAE(32, 64, [128, 256], [7,7], [2,2], [0.1,0.1], 128, 2, 128, 
               condition_hidden=False, condition_output=True)

ints = torch.randint(0, 31, (8, 10))
x = ints[:,:-1]
y = ints[:,1:]

_ = vae(x)

_ = vae.sample(8, 16)

z = vae.prior.sample([8])
_ = vae.sample(8, 16, z=z)

_ = vae.get_rl_tensors(x,y)

In [ ]:
# export
        
class MLP_VAE(VAE):
    def __init__(self, d_vocab, d_embedding, encoder_d_in, encoder_dims, encoder_drops,
                 d_hidden, n_layers, d_latent, dec_drop=0., 
                 condition_hidden=True, condition_output=True,
                 prior=None, bos_idx=0):
        
        encoder = MLP_Encoder(encoder_d_in, encoder_dims, d_latent, encoder_drops)
        
        decoder = Conditional_LSTM_Block(d_vocab, d_embedding, d_hidden, d_embedding,
                                d_latent, n_layers, lstm_drop=dec_drop, lin_drop=dec_drop, 
                                condition_hidden=condition_hidden, condition_output=condition_output)
        
        super().__init__(encoder, decoder, prior, bos_idx)
    
    

In [ ]:
vae = MLP_VAE(32, 64, 128, [64, 32], [0.1, 0.1], 128, 2, 128, 
               condition_hidden=False, condition_output=True)

ints = torch.randint(0, 31, (8, 10))
x = ints[:,:-1]
y = ints[:,1:]

condition = torch.randn((8,128))


_ = vae(condition, x)

_ = vae.sample(8, 16)

z = vae.prior.sample([8])
_ = vae.sample(8, 16, z=z)

_ = vae.get_rl_tensors([condition,x],y)

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()